# Part 1: Gathering relevant links

This notebook is teh first of three used to collect and format data from select retailers about their website messages about covid-19 mitigation at their stores for customers and/or employees. 

We will first install a python package known as scrapy to help us go from page to page on each of the retailers. Then we will use a list of retailers website urls to search for *covid* and *coronavirus* related pages on the retailers. 

We will establish a limit to our search in 2 ways. The first of these is setting a depth limit to 1. This means that we will follow links from the first website url to secondary pages. However, we will not follow links on the secondary pages. This keeps us from visiting too many pages of unimportance. The second limit will be on the number of pages we'll consider for each retailer. This value is set to 40 pages, but can be adjusted if it appears important pages aren't getting found. Unfortunately the only way to determine this is if you either notice limited amounts of scrapes for that retailer in the final visualizations, or manually check on the retailers page and identify important urls that weren't caught. If this happens feel free to add those retailer urls to the pass 1 list.   

A general example of the first limit is as follows. Imagine a retailer has Pages 1, 2, 3. In order to get to Page 3 you must do Page 1 -> Page2 -> Page 3. The "->"s represent levels of depth here, so Page 1 -> Page 2 is allowed because it is depth 1, but not Page 1 -> Page 2 -> Page 3 because it is depth 2. 

You might ask "then how do we get to Page 3 if we really need to"?. To accomplish this we will run the collection 2 times. The first time is simply to gather the "low hanging fruit" pandemic related pages. These pages will be added to our list of retailer urls to look through, and upon the second pass we will follow additional links that may be relevant. Lets look at an example

## Pass 1 list of links
https://www.walgreens.com/

https://www.cvs.com/

...

## Pass 2 list of links
https://www.walgreens.com/

https://www.walgreens.com/findcare/covid19/testing?ban=covidfy21_vaccine_testing_brandstory_12.09.20

...

https://www.cvs.com/

https://www.cvs.com/content/coronavirus?icid=cvs-home-hero1-banner1-link2-coronavirus-resources

...

## Final set of links
https://www.walgreens.com/

https://www.walgreens.com/findcare/covid19/testing?ban=covidfy21_vaccine_testing_brandstory_12.09.20

https://www.labcorp.com/coronavirus-disease-covid-19/news

...

https://www.cvs.com/

https://www.cvs.com/content/coronavirus?icid=cvs-home-hero1-banner1-link2-coronavirus-resources

https://www.cvs.com/minuteclinic/covid-19-testing

...

The final set of links will be passed along to the part 2 notebook so that we can collect wayback machine data about what content may have been on these pages, or when they were created.

# Getting started

Lets make sure we have all the python libraries that will help us with our collections.


first step here is to install scrapy to help us with navigating from website to website.

Also make sure to get the superbly named BeautifulSoup. 

Then we get the tqdm package which lets us create fun progress bars for things.


Besides those everything we will use should be in the standard library.



Now we will import various python packages that we need to make our code work

In [5]:

import scrapy
import re
import multiprocessing as mp
from multiprocessing import Process,Queue
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup as bs
import bs4

# Getting to the code

Here's where we define the functions that we will use for our link fetching.



# Main part

This is the bulk of our notebook here. This function returns a spider class that is setup to use the urls we pass to the function. 

The high level view of what the class does is as follows. This spider explores a website automatically only going as far as 1 level of depth in the links, and only 40 total pages explored for each retailer. Only pages that contain *covid* or *coronavirus* in their links will be collected. This is all specified to help make the scraping process more efficient. In theory some results could be missed because we aren't mirroring the entire website, but in comparing the results with manual inspection of pages, it appears we uncover the important links that are most apparent to viewers of the webpages. It should be mentioned also that mirroring is unwise when retailers such as amazon and costco are included as their websites include a tremendous amount of pages.

In [6]:

def url_make_spider(url,retailer):
    # urls gets used in start_requests
    class MySpider(scrapy.Spider):
        # Your spider definition
        # the name isn't critical for anything
        name='retail'
        # this is a counter of how many pages we've visited
        count = 0
        # this is the cap on the number of pages we've hit
        limit = 200
        # store urls related to covid and coronavirus
        important_urls=[]
        # any other links go in here
        # other modifications to this class could include visiting these links if we run out of important urls to visit
        other_urls=[]
        # settings to be polite web scrapers,
        # throttle the speed somewhat
        # don't log unneccessary stuff
        # include a 1 second delay in between each page
        # don't go further than 1 level of depth
        custom_settings = {
            'AUTOTHROTTLE_ENABLED': True,
            "LOG_LEVEL":"CRITICAL",
            'DOWNLOAD_DELAY': 1,
            'DEPTH_LIMIT': 1,
        }
        # use the parameter urls from the url_make_spider function to start our spider 
        def start_requests(self):
            yield scrapy.Request(url=url,callback=self.parse)
        # look through the data on the page
        def parse(self,response):
            # save any of the covid info it finds
            # convert to bs
            # this makes it possible to convert a websites' html to just what would be visible text
            soup = bs(response.body)
            # navigate the tags
            relevant_info = []
            for tag in soup.find_all(True):
                # skip tags that don't represent important visual references on the website to thhe pandemic
                if len(tag.contents) > 0 and  not tag.name in "a style script".split(" "):
                    #first see if thhere's an inner string , then see if pandemic terms are in it?
                    if bs4.element.NavigableString == type(tag.contents[0]):
                        content = tag.contents[0].lower()
                        if 'covid' in content or 'coronavirus' in content:
                            relevant_info.append(content)
            if len(relevant_info) > 0:
                yield {"website":response.url,"retailer":retailer,"text":relevant_info}
            # loop through all the links on the page
            #prefer the pandemic related links first

            for a in response.css('a'):
                a_str = a.get().lower()
                if 'covid' in a_str or 'coronavirus' in a_str:
                    self.important_urls.append(a)
                else:
                    self.other_urls.append(a)
            #ensure we go in breadth first order
            # this means if a link was added to our list, we should visit it before others that came after
            # treat the important_urls as a FIFO (first in first out) queue
            while len(self.important_urls) > 0 and self.count < self.limit:
                self.count += 1
                yield response.follow(self.important_urls.pop(0),callback=self.parse)
            if len(self.important_urls) == 0:
                while len(self.other_urls) > 0 and self.count < self.limit:
                    self.count += 1
                    yield response.follow(self.other_urls.pop(0),callback=self.parse)
    # give back the class for use in the crawl process        
    return MySpider



This function takes a url in and starts a spider to derive data from that retailer url. It appends all the relevant results to the `results.jl` file. Each line of this file looks like this
```
{"website":"https://www.retailer.com/covid-updates","retailer":retailer,"text":["...covid-19 is bad...",...]
```
you can see the retailer url and the entire website's snippets mentioning covid.

In [2]:

def run_spider(url,retailer):
    # user agent is the thing webpage servers see to identify our spider
    # feeds is the output for our spider
    process = CrawlerProcess({
            'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1), retail covid research',
            "FEEDS":{
                "./data_processing/all_urls.jl":{"format":"jsonlines"},
        }
    })
    # run the function we defined above using the url we are given
    myspider_class = url_make_spider(url,retailer)
    # set the crawl function with the spider class we got from running url_make_spider
    process.crawl(myspider_class)
    #start the process
    process.start() # the script will block here until the crawling is finished
    print("run_spider done")


This function `go` is helpful if we decide to run up to 4 spiders at the same time. `qu` is a queue with retailer urls in it. this function draws from the queue and starts spiders with the provided url until there's no more items in the `qu`

In [3]:
def go(qu):
    while not qu.empty():
        url,retailer = qu.get()
        print("going")
        p = Process(target=run_spider,args=(url,retailer))
        p.start()
        p.join()
        print("another go done")
    print("queue empty")

Now we actually run the collection of our links from all the different pages. Feel free to add other retailers to this list, or remove them. But make sure to provide lines in the following pattern. 

`retailer_url,retailer_name`

like

`https://www.safeway.com,safeway`

the %%capture at the top prevents the cell from creating output. It can be desirable to not show the output for code like ours because it creates so much output the browser starts to slow down and freeze. If this happens to you remove the `#` ahead of the %%capture, and re-run the cell.

you may also choose to close the browser tab. This won't stop the code from running in the background. Just know that when you open the notebook again, you may need to run certain cells again if you want to continue. Cells like the imports cell, function definition cells, or cells containing variables that unexecuted code depends on will need to be re executed.  

In [4]:
import multiprocessing as mp 
cpus = mp.cpu_count()
cpus

72

In [7]:
#%%capture
retailers = """
https://www.publix.com/,publix
https://www.riteaid.com/,riteaid
https://corporate.lowes.com/,lowes
https://www.lowes.com/,lowes
https://www.albertsons.com,albertsons
https://www.aldi.us,aldi
https://www.safeway.com,safeway
https://www.frysfood.com,frys
https://www.amazon.com/,amazon
https://www.wholefoodsmarket.com/,wholefoods
https://www.cvs.com/,cvs
https://www.dollargeneral.com/,dollargeneral
https://www.dollartree.com/,dollatree
https://corporate.walmart.com/galleries/neighborhood-market,walmart
https://www.walmart.com/c/brand/marketside,walmart
https://www.aholddelhaize.com/en/home/,aholddelhaize
https://www.samsclub.com/,samsclub
https://www.target.com/,target
https://corporate.target.com/,target
https://www.homedepot.com/,homedepot
https://corporate.homedepot.com/,homedepot
https://ir.homedepot.com/,homedepot
https://www.kroger.com/,kroger
https://www.traderjoes.com/,traderjoes
https://www.walgreens.com/,walgreens
https://www.walgreensbootsalliance.com/,walgreens
https://www.walmart.com/,walmart
https://www.costco.com/,costco
https://www.fredmeyer.com/,fredmeyer
""".strip().split("\n")

## test with only a few of them
q = Queue()
for retailer in retailers:
    q.put(retailer.split(","))

procs =[]
cpus = mp.cpu_count()
if cpus > len(retailers):
    cpus = len(retailers)
for i in range(cpus):
    p = Process(target=go,args=(q,))
    p.start()
    procs.append(p)
for p in procs:
    p.join()

print("done")

going
going
going
going


2020-12-17 00:41:22 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-17 00:41:22 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-17 00:41:22 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.5 | packaged by conda-forge | (default, Jul 24 2020, 01:25:15) - [GCC 7.5.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 3.0, Platform Linux-4.15.0-126-generic-x86_64-with-glibc2.10
2020-12-17 00:41:22 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-12-17 00:41:22 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'DEPTH_LIMIT': 1,
 'DOWNLOAD_DELAY': 1,
 'LOG_LEVEL': 'CRITICAL',
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1), retail '
               'covid research'}
2020-12-17 00:41:22 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0,

run_spider done
another go done
queue empty
run_spider done
another go done
queue empty
run_spider done
another go done
queue empty
run_spider done
another go done
queue empty
done


second part, optional, if ~120 links isn't enough for you.

This sets the result of our first pass as the set of pages to explore for meaningful links. This gets around our depth restriction as explained at the top of the page. This second pass has taken up to an hour for me to complete, it also appears to increase the number of links to search history on by about 4x as many, up to ~590 unique links relating to modern day pandemic retailer website info. 


In [21]:
#%%capture
import json
with open("./data_processing/all_urls.jl","r") as phile:
    moreretailers = []
    for line in phile:
        moreretailers.append(json.loads(line))
        
retail_websites = [[e["website"],e["retailer"]] for e in moreretailers]
q = Queue()
for retailer in retail_websites:
    q.put(retailer)

procs =[]
for i in range(4):
    p = Process(target=go,args=(q,))
    p.start()
    procs.append(p)
for p in procs:
    p.join()

goinggoing

going
going


2020-12-16 13:55:01 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-16 13:55:01 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-16 13:55:01 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-16 13:55:01 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-16 13:55:01 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.3 (default, May 19 2020, 18:47:26) - [GCC 7.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Linux-4.15.0-126-generic-x86_64-with-glibc2.10
2020-12-16 13:55:01 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.3 (default, May 19 2020, 18:47:26) - [GCC 7.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Linux-4.15.0-126-generic-x86_64-with-glibc2.10
2020-1

going


2020-12-16 13:55:02 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-16 13:55:02 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.3 (default, May 19 2020, 18:47:26) - [GCC 7.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Linux-4.15.0-126-generic-x86_64-with-glibc2.10
2020-12-16 13:55:02 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-12-16 13:55:02 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'DEPTH_LIMIT': 1,
 'DOWNLOAD_DELAY': 1,
 'LOG_LEVEL': 'CRITICAL',
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1), retail '
               'covid research'}


going


...

2020-12-16 14:54:02 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-16 14:54:02 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.3 (default, May 19 2020, 18:47:26) - [GCC 7.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Linux-4.15.0-126-generic-x86_64-with-glibc2.10
2020-12-16 14:54:02 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-12-16 14:54:02 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'DEPTH_LIMIT': 1,
 'DOWNLOAD_DELAY': 1,
 'LOG_LEVEL': 'CRITICAL',
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1), retail '
               'covid research'}


going


2020-12-16 14:54:15 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-16 14:54:15 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.3 (default, May 19 2020, 18:47:26) - [GCC 7.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Linux-4.15.0-126-generic-x86_64-with-glibc2.10
2020-12-16 14:54:15 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-12-16 14:54:15 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'DEPTH_LIMIT': 1,
 'DOWNLOAD_DELAY': 1,
 'LOG_LEVEL': 'CRITICAL',
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1), retail '
               'covid research'}


going


2020-12-16 14:54:16 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-16 14:54:16 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.3 (default, May 19 2020, 18:47:26) - [GCC 7.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Linux-4.15.0-126-generic-x86_64-with-glibc2.10
2020-12-16 14:54:16 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-12-16 14:54:16 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'DEPTH_LIMIT': 1,
 'DOWNLOAD_DELAY': 1,
 'LOG_LEVEL': 'CRITICAL',
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1), retail '
               'covid research'}


In [23]:
uniques = set()
with open("./data_processing/all_urls.jl","r") as phile:
    for line in phile:
        uniques.add(json.loads(line)["website"])
        
len(uniques)

596